# Master TIDE - Conférences Python 2020

Francis Wolinski

&copy; 2020 Yotta Conseil

In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#no warning
import warnings
warnings.simplefilter("ignore")

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_colwidth', 100)

## Vector space model avec scikit-learn TF-IDF

En recherche d'information, `TF–IDF`, abréviation de **term frequency–inverse document frequency**, est un indicateur statistique qui cherche à capturer l'importance d'un terme dans un document au sein d'un corpus.

Un corpus contient `N` documents. Un terme apparaît dans `n` documents du corpus. pour un document donné de longueur `L`, un terme apparaît `T` fois.

`TF–IDF` est le produit de 2 facteurs :
- *term frequency* (dans un document donné) :
    - $\frac{T}{L}$
- *inverse document frequency* (fréquence d'apparition dans le corpus):
    - $\log(\frac{N}{n})$
    
Dans ce modèle, chaque document est représenté par un vecteur normalisé componsé du calcul `TF–IDF` pour chaque terme qu'il contient (vector space model).

La librairie `scikit-learn` permet de calculer les vecteurs `TF–IDF` pour un corpus de documents.

Cette librairie comprend différentes classes traitant des textes:

- `CountVectorizer`: implémente la tokenisation et le décompte des occurrences
- `TfidfTransformer`: transforme une matrice de comptage en une représentation `TF` ou `TF–IDF`
- `TfidfVectorizer`: équivalent à *CountVectorizer* suivi par *TfidfTransformer*

Ces classes produisent des **matrices creuses**, c'est-à-dire des matrices pleines de 0 à 99% ou 99.9%, puisque chaque document ne contient que *100* ou *1000* termes et que l'ensemble d'un corpus peut contenir plusieurs *100K* termes.

*Nota bene*

- Traiter des matrices creuses nécessite une certaine attention car il est générallement pas possible de les transformer en *numpy* arrays ou *pandas* dataframes, à cause des limitations de mémoire.

<div class="alert alert-warning">
<b>Pour plus d'information</b>
<ul>
    <li><a href='https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction'>https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction</a></li>
</ul>
</div>

In [ ]:
# load dataset
df = pd.read_csv('imdb_master.csv',
                 encoding='latin-1',
                 usecols=['review', 'label'])
df.shape

In [ ]:
# head
df.head()

In [ ]:
# remplacement des charactères mal encodés
df['review'] = df['review'].apply(lambda x: x.replace('Ã¡', 'á'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã ', 'à'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã ', 'à'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¢', 'â'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã\xa0', 'à'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¥', 'å'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã£', 'ã'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã»', 'â'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã§', 'ç'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã©', 'é'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¨', 'è'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã«', 'ë'))
df['review'] = df['review'].apply(lambda x: x.replace('Ãª', 'ê'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¯', 'ï'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã®', 'î'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¬', 'ì'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã\xad', 'í'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã±', 'ñ'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã³', 'ó'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã²', 'ò'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¶', 'ö'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã´', 'ô'))
df['review'] = df['review'].apply(lambda x: x.replace('Ãµ', 'õ'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã°', 'ð'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¸', 'ø'))
df['review'] = df['review'].apply(lambda x: x.replace('Ãº', 'ú'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¹', 'ù'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¼', 'ü'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã½', 'ý'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¿', 'ÿ'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã?', 'Æ'))
df['review'] = df['review'].apply(lambda x: x.replace('Ã¦', 'æ'))
df['review'] = df['review'].apply(lambda x: x.replace('Â', ''))
# br
df['review'] = df['review'].apply(lambda x: x.replace('<br />', ' '))

### Alignement de textes

In [ ]:
# find word
def find(word):
    selection = df.loc[df['review'].str.contains(word, regex=False)].head(30)
    result = selection['review'].apply(lambda x: (' ' * 25 + x + ' ' * 25)[x.find(word):x.find(word) + 50 + len(word)])
    return result

In [ ]:
find('Hitchcock')

### Traitement "manuel" des textes

In [ ]:
# traitement "manuel"
import re
from collections import Counter

rx = re.compile('[A-Za-zÀ-ÿ0-9]+')

c = Counter()
df['review'].apply(lambda x: c.update(rx.findall(x)))

c

In [ ]:
# vocabulaire
vocab = pd.Series(list(c.values()), index=c.keys())
vocab = vocab.sort_values(ascending=False)
vocab.head(30)

<div class="alert alert-success">
<b>Exercise 1</b>
<ul>
<li>Calculer un nouveau dictionnaire fréquenciel en comptabilisant chaque terme une seule fois par document et en mettant les termes en minuscules</li>
</ul>
</div>

### CountVectorizer

`CountVectorizer`: implémente la tokenisation des termes et le comptage des occurrences.

Il effectue le même calcul que `Counter`. On peut calculer facilement le nombre de fois que chaque terme apparaît dans un document, ou dans tout le corpus.

In [ ]:
# import
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
%%time
# CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['review'])
X

On obtient une matrice creuse de *100,000* x *142,189*, c'est-à-dire, 142 milliards de nombres, mais qui ne contient que *13,681,951* éléments:

En fait, on a :
- 100,000: documents
- 142,189: termes
- 13,681,951: éléments

In [ ]:
# densité & sparsité
density = X.count_nonzero() / X.shape[0] / X.shape[1]
print('density:', density)
sparsity = 1.0 - density
print('sparsity: {:.3f}%'.format(sparsity))

In [ ]:
# memory error!
X.toarray()

Il est possible d'utiliser la méthode `get_feature_names()` de `CountVectorizer` qui donne la liste de tous les termes du corpus, dans le même ordre que la matrice creuse.

In [ ]:
# termes d'un document donné
big_review_id = df['review'].str.len().idxmax()
# ligne de la matrice avec les termes
values = X[big_review_id].toarray()[0]
# liste de tous les termes
words = vectorizer.get_feature_names()
# mise en Series
vocab = pd.Series(values, index=words)
# nombre total de termes et nombre de termes dans le document
print(len(vocab), len(vocab[vocab!=0]))
vocab = vocab[vocab!=0]
vocab = vocab.sort_values(ascending=False)
vocab.head(30)

In [ ]:
# termes de l'ensemble des documents en sommant selon l'axis 0
# somme en colonne par terme
values = np.asarray(X.sum(axis=0))[0]
# liste de tous les termes
words = vectorizer.get_feature_names()
# mise en Series
vocab = pd.Series(values, index=words)
vocab = vocab.sort_values(ascending=False)
vocab.head(30)

### TfidfVectorizer

In [ ]:
# import
from sklearn.feature_extraction.text import TfidfVectorizer

In order to get rid off stop words, rare words and unknown words, we use the `TfidfVectorizer` with few dedicated parameters. The result is a smaller sparse matrix which is still not possible to cast to array.

In [ ]:
# stop words from nltk
if False:
    from nltk.corpus import stopwords
    stopwords_en = set(stopwords.words("english"))
else:
    import json
    with open('stopwords_en.json') as f:
        stopwords_en = set(json.load(f))
stopwords_en

In [ ]:
%%time
# TfidfVectorizer
pattern = '[A-Za-zÀ-ÿ0-9]+(?:\'(?:d|ll|re|s|t|ve))?'
vectorizer = TfidfVectorizer(analyzer='word',
                             token_pattern=pattern,
                             min_df=10,
                             stop_words=stopwords_en)
X = vectorizer.fit_transform(df['review'])
X

In [ ]:
# density & sparsity
density = X.count_nonzero()/X.shape[0]/X.shape[1]
print('density:', density)
sparsity = 1.0 - density
print('sparsity: {:.3f}%'.format(sparsity))

Il est possible de calculer la somme des `TD-IDF` de chaque terme pour l'ensemble du corpus.

In [ ]:
# termes de tous les documents en sommant selon l'axis 0
# somme en colonnes
values = np.asarray(X.sum(axis=0))[0]
index = vectorizer.get_feature_names()
vocab = pd.Series(values, index=index)
vocab = vocab.sort_values(ascending=False)
vocab.head(30)

Nous allon sutiliser la vectorisation des documents avec `TF-IDF` pour rechercher les documents similaires à un document donné.

In [ ]:
# sélection d'un document dans le corpus
review_id = 13473
df.loc[review_id, 'review']

In [ ]:
# indices des termes présents dans ce document
indices = np.nonzero(X[review_id].toarray())[1]
indices

In [ ]:
# termes correspondants
termes = [vectorizer.get_feature_names()[i] for i in indices]
termes

In [ ]:
# valeur TF-IDF de chaque terme
values = [X[review_id, i] for i in indices]
values

In [ ]:
# mise en Series
s = pd.Series(values, index=termes)
s = s.sort_values(ascending=False)
s

Pour trouver les documents similaires, nous calculons le produit scalaire entre le vecteur de chacun des documents et le vecteur du document sélectionné, on conserve les documents qui produisent les meilleurs scores.

In [ ]:
# produit scalaire du document donné avec lui-même
np.dot(X[review_id], X[review_id].T).toarray()[0]

In [ ]:
# indices des 10 meilleurs documents
ids = np.dot(X, X[review_id].T).toarray().T[0].argsort()[-10:]
ids

In [ ]:
# textes asociés
df.loc[ids]

Il est possible de tout mettre dans une seule fonction qui produit les n meilleurs documents.

In [ ]:
# fonction qui trouve les revues similaires
def find_similar_reviews(review_id, n=5):
    # print review
    print(df.loc[review_id, 'review'])
    print()
    # ids des termes de la review
    word_ids = np.nonzero(X[review_id].toarray())[1]
    # termes de la review
    words = [vectorizer.get_feature_names()[word_id] for word_id in word_ids]
    # TF-IDF des termes de la review
    values = [X[review_id, word_id] for word_id in word_ids]
    # mises en Series et print
    s = pd.Series(values, index=words)
    s = s.nlargest(20)
    print(s)
    print()
    # calcul du produit matriciel entre la matrice et la revue
    # on retient du n° 2 au n° n+1
    ids = np.dot(X, X[review_id].T).toarray().T[0].argsort()[-n-1:-1]
    # collecte des reviews et print
    reviews = df.loc[ids[::-1], 'review']
    for i in range(len(reviews)):
        print(i+1)
        print(reviews.iloc[i])
        print()

In [ ]:
# test
find_similar_reviews(review_id)

In [ ]:
# random review
review_id = np.random.randint(100_000)
find_similar_reviews(review_id)

Les autres applications de `TF-IDF` sont la classification, le clustering et l'extraction de thématiques de documents.

## 2.2 Gensim Word2Vec (démo)

**Gensim** : topic modelling for humans

Hypothèses :
- un corpus composé de documents
- les documents sont composés de mots

L'algorithme utilise le plongement lexical (word embeddings) (i.e. le contexte local) pour produire des vecteurs pour chacun des mots :

- préprocessing des documents : `gensim.utils.simple_preprocess()`
- modélisation des mots : `gensim.models.Word2Vec()`

Ensuite, il est possible d'utiliser les propriétés algébriques des vecteurs pour calculer et utiliser la similarité entre des mots. L'utilisation typique de `gensim` est d'aider à trouver des synonymes ou des antonymes pour des applications dédiées de text mining.

- `most_similar(positive=[...])`: trouve les entités similaires à une liste d'entités (synonymes)
- `most_similar(positive=[...], negative=[...])`: trouve les entités similaires à une liste d'entités (synonyms) et celles qui sont le moins similaires à une autre (antonymes)
- `closer_than(entity1, entity2)`:  trouve les entités plus proches de `entity1` que `entity2` l'est de `entity1`
- `doesnt_match()`: trouve l'entité qui correspond le moins au autres

Il y a beaucoup d'autres techniques du type "X2Vec" : e.g., Doc2Vec, Fact2Vec.

Installation, ouvrir un terminal dans le dossier `anaconda3/condabin` :

- PC: `.\conda install -c anaconda gensim`
- Mac: `./conda install -c anaconda gensim`

<div class="alert alert-warning">
<b>Further reading</b>
<ul>
    <li><a href='https://radimrehurek.com/gensim/index.html'>https://radimrehurek.com/gensim/index.html</a></li>
</ul>
</div>

In [ ]:
# import

import gensim

#### Dataset

In [ ]:
# loading data
df = pd.read_csv('imdb_master.csv',
                 encoding='latin-1',
                 usecols=['review'])
df.head()

#### Application de l'algorithme de `gensim `

In [ ]:
%%time
# preprocesing & modelling
if False:
    # step 1: preprocessing texts
    documents = df['review'].apply(lambda x: gensim.utils.simple_preprocess(x)).values
    # step 2: process documents
    # time depends on vector size, here 150
    model = gensim.models.Word2Vec(documents,
                              size=150,
                              window=10,
                              min_count=2,
                              workers=4)
    model.save('imdb_master.bin')
else:
    model = gensim.models.Word2Vec.load('imdb_master.bin')

#### Utilisation du modèle

La fonction `most_similar()` prend des mots positifis et/ou negatifs, afin de trouver des mots similaires, ou non similaires.

C'est utile pour fabriquer des liste de synonymes ou d'antonymes pour une application de text mining.

**Synonymes**

In [ ]:
# good
model.wv.most_similar(positive='good')

In [ ]:
# great
model.wv.most_similar(positive='great')

In [ ]:
# bad
model.wv.most_similar(positive='bad')

In [ ]:
# horrible
model.wv.most_similar(positive='horrible')

In [ ]:
# superhero
model.wv.most_similar(positive=['superhero'])

In [ ]:
# hitchcock
model.wv.most_similar(positive=['hitchcock'])

In [ ]:
# hitchcock & french
model.wv.most_similar(positive=['hitchcock', 'french'])

**Arithmétique des mots**

In [ ]:
# +woman +king -man
model.wv.most_similar(positive=['king', 'woman'], negative=['man'])

In [ ]:
# +man +queen -woman
model.wv.most_similar(positive=['queen', 'man'], negative=['woman'])

In [ ]:
model.wv.most_similar(positive=['actor', 'woman'], negative=['man'])

**Trouver l'intrus**

In [ ]:
# doesnt_match
model.wv.doesnt_match(['thriller', 'drama', 'comedy', 'melodrama', 'romance'])

## WordCloud (démo)

Installation, ouvrir un terminal dans le dossier `anaconda3/condabin` :

- sur PC : `.\conda install -c conda-forge wordcloud`
- sur Mac : `./conda install -c conda-forge wordcloud`

Wordcloud est une librairie Python qui permet de fabriquer des nuages de mots. La taille des mots dépend de leur nombre d'occurences.

In [ ]:
# import et utilisation de wordcloud
from wordcloud import WordCloud

# un texte
text = df.loc[df['review'].str.len().idxmax(), 'review']

# texte aléatoire
#from numpy import random
#random.seed(100)
#text = df.sample(1)['review'].values[0]

# à essayer avec différentes options, e.g. stopwords=[]
wc = WordCloud(max_words=100,
               stopwords=stopwords_en,
               normalize_plurals=True,
               width=800,
               height=600)
wc.generate(text)

# affichage du nuage de mots
plt.figure(figsize=(12,9))
plt.axis("off")
plt.imshow(wc);